In [1]:
import sys
sys.path.append("..")
import pickle
import yaml
import os
import re
import sys
import numpy as np
import pandas as pd
import argparse
import torch
import datetime
import json
import yaml
import os
import sys
from scipy import optimize
from torch.utils.data import DataLoader, Dataset
from data_loaders import *
import missing_process.missing_method as missing_method
from missing_process.block_rules import *

In [156]:
def MCAR(observed_values, missing_ratio, masks):
    for col in range(observed_values.shape[1]):  # col #

        obs_indices = np.where(observed_values[:, col])[0]
        miss_indices = np.random.choice(
        obs_indices, (int)(len(obs_indices) * missing_ratio), replace=False
        )
        masks[miss_indices, col] = False

    return masks

def process_func(dataname, path: str, aug_rate=1,missing_type = "MCAR",
                  missing_para = ""):
 
    data = dataset_loader(dataname)
    # print(data)
    # data.replace("?", np.nan, inplace=True)
    # Don't apply data argument (use n*dataset)
    # data_aug = pd.concat([data] * aug_rate)

    observed_values = data["data"].astype("float32")

    observed_masks = ~np.isnan(observed_values)
    masks = observed_masks.copy()

    "Need input origin dataset and parameters"
    if missing_type == "MCAR":
        masks = MCAR(observed_values,missing_para,masks)

    elif missing_type == "quantile":
        Xnan, Xz = missing_method.missing_by_range(observed_values, missing_para)
        masks = np.array(~np.isnan(Xnan), dtype=np.float)

    elif missing_type == "logistic":
        masks = missing_method.MNAR_mask_logistic(observed_values, missing_para)

    elif missing_type == "diffuse":
        #masks = missing_method.MNAR_self_mask_logistic(observed_values, missing_para)

        masks = diffuse_mnar_single(observed_values, missing_para[0],missing_para[1])


    # gt_mask: 0 for missing elements and manully maksed elements
    gt_masks = masks.reshape(observed_masks.shape)

    observed_values = np.nan_to_num(observed_values)
    observed_masks = observed_masks.astype(int)
    gt_masks = gt_masks.astype(int)

    return observed_values, observed_masks, gt_masks, data["data"].shape[1]

In [151]:
class tabular_dataset(Dataset):
    # eval_length should be equal to attributes number.
    def __init__(
        self, dataname, use_index_list=None, 
        aug_rate=1, seed=0,
        missing_type = "MCAR", missing_para = "",missing_name = "MCAR"
        ):
        #self.eval_length = eval_length
        np.random.seed(seed)
        
        dataset_path = f"datasets/{dataname}/data.csv"
        processed_data_path = (
            f"datasets/{dataname}/{missing_type}-{missing_name}_seed-{seed}.pk"
        )
        processed_data_path_norm = (
            f"datasets/{dataname}/{missing_type}-{missing_name}_seed-{seed}_max-min_norm.pk"
        )
        # If no dataset created
        if not os.path.isfile(processed_data_path):
            self.observed_values, self.observed_masks, self.gt_masks, self.eval_length = process_func(
                dataname, dataset_path, aug_rate=aug_rate,
                missing_type = missing_type, missing_para = missing_para
            )
            print("self.eval_length",self.eval_length)
            with open(processed_data_path, "wb") as f:
                pickle.dump(
                    [self.observed_values, self.observed_masks, self.gt_masks, self.eval_length], f
                )
            print("--------Dataset created--------")

        elif os.path.isfile(processed_data_path_norm):
            with open(processed_data_path_norm, "rb") as f:
                self.observed_values, self.observed_masks, self.gt_masks, self.eval_length = pickle.load(
                    f
                )
            print("--------Normalized dataset loaded--------")
        
        if use_index_list is None:
            self.use_index_list = np.arange(len(self.observed_values))
        else:
            self.use_index_list = use_index_list

    def __getitem__(self, org_index):
        index = self.use_index_list[org_index]
        s = {
            "observed_data": self.observed_values[index],
            "observed_mask": self.observed_masks[index],
            "gt_mask": self.gt_masks[index],
            "timepoints": np.arange(self.eval_length),
        }
        return s

    def __len__(self):
        return len(self.use_index_list)

In [4]:
def get_dataloader(dataname, seed=1, nfold=5, batch_size=16,
                   missing_type = "MCAR", missing_para = "", missing_name = "MCAR"):

    dataset = tabular_dataset(dataname = dataname,seed=seed,
                              missing_type = missing_type, missing_para = missing_para,
                                missing_name = missing_name)
    print(f"Dataset size:{len(dataset)} entries")
    
    
    indlist = np.arange(len(dataset))

    np.random.seed(seed + 1)
    np.random.shuffle(indlist)

    tmp_ratio = 1 / nfold
    start = (int)((nfold - 1) * len(dataset) * tmp_ratio)
    
    end = (int)(nfold * len(dataset) * tmp_ratio)

    test_index = indlist[start:end]
    remain_index = np.delete(indlist, np.arange(start, end))

    np.random.shuffle(remain_index)

    # Modify here to change train,valid ratio
    num_train = (int)(len(remain_index) * 0.9)
    train_index = remain_index[:num_train]
    valid_index = remain_index[num_train:]



    # Here we perform max-min normalization.
    processed_data_path_norm = (
        f"datasets/{dataname}/{missing_type}-{missing_name}_seed-{seed}_max-min_norm.pk"
    )
    if not os.path.isfile(processed_data_path_norm):
        print(
            "--------------Dataset has not been normalized yet. Perform data normalization and store the mean value of each column.--------------"
        )
        # data transformation after train-test split.
        col_num = dataset.observed_values.shape[1]
        max_arr = np.zeros(col_num)
        min_arr = np.zeros(col_num)
        mean_arr = np.zeros(col_num)
        for k in range(col_num):
            # Using observed_mask to avoid counting missing values.
            obs_ind = dataset.observed_masks[train_index, k].astype(bool)
            temp = dataset.observed_values[train_index, k]
            max_arr[k] = max(temp[obs_ind])
            min_arr[k] = min(temp[obs_ind])
        print(f"--------------Max-value for each column {max_arr}--------------")
        print(f"--------------Min-value for each column {min_arr}--------------")

        dataset.observed_values = (
            (dataset.observed_values - 0 + 1) / (max_arr - 0 + 1)
        ) * dataset.observed_masks

        with open(processed_data_path_norm, "wb") as f:
            pickle.dump(
                [dataset.observed_values, dataset.observed_masks, dataset.gt_masks, dataset.eval_length], f
            )

    # Create datasets and corresponding data loaders objects.
    train_dataset = tabular_dataset(dataname = dataname,
        use_index_list=train_index, seed=seed,
        missing_type = missing_type, missing_para = missing_para, missing_name = missing_name
    )
    #train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=1)
    valid_dataset = tabular_dataset(dataname = dataname,
        use_index_list=valid_index, seed=seed,
        missing_type = missing_type, missing_para = missing_para, missing_name = missing_name
    )
    #valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=0)

    test_dataset = tabular_dataset(dataname = dataname,
        use_index_list=test_index, seed=seed,
        missing_type = missing_type, missing_para = missing_para, missing_name = missing_name
    )
   #test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=0)

    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(valid_dataset)}")
    print(f"Testing dataset size: {len(test_dataset)}")

    #return train_loader, valid_loader, test_loader


# cc : self_mask

# Red: Self_mask

# white: Self_mask

In [149]:
def diffuse_mnar_single(data, up_percentile = 0.5, obs_percentile = 0.5):
    
    def scale_data(data):
        min_vals = np.min(data, axis=0)
        max_vals = np.max(data, axis=0)
        scaled_data = (data - min_vals) / (max_vals - min_vals)
        return scaled_data

    data = scale_data(data)

    mask = np.ones(data.shape)

    n_cols = data.shape[1]
    n_miss_cols = int(n_cols * 0.5)  # 选择50%的列作为缺失列
    miss_cols = np.random.choice(n_cols, size=n_miss_cols, replace=False)  # 随机选择缺失列的索引

    obs_cols = [col for col in range(data.shape[1]) if col not in miss_cols]
    
    for miss_col in miss_cols:
        missvar_bounds = np.quantile(data[:, miss_col], up_percentile)
        temp = data[:, miss_col] > missvar_bounds
        
        obsvar_bounds = np.quantile(data[temp][:, obs_cols], obs_percentile)
        temp2 = data[:, miss_col] > obsvar_bounds

        merged_temp = np.logical_or(temp, temp2).astype(int)
        mask[:, miss_col] = merged_temp
    print("Missing Rate",1 - np.count_nonzero(mask) / mask.size)
    return mask

In [161]:
dataset = "banknote"#"concrete_compression","wine_quality_white","banknote"
seed = 1
nfold = 5
missingtype = "diffuse"
#missingtype = "self_mask"

missing_rule = load_json_file("diffuse_ratio.json")


for rule_name in missing_rule:
    rule = missing_rule[rule_name]
    print("Current Rule",rule )
    # Create folder
    # Every loader contains "observed_data", "observed_mask", "gt_mask", "timepoints"
    get_dataloader(
        dataname=dataset,
        seed=seed,
        nfold=nfold,
        batch_size=128,
        missing_type = missingtype,
        missing_para = rule,
        missing_name = rule_name
    )

Current Rule [0.25, 0.25]
Goto diffuse
Missing Rate 0.08789204959883301
self.eval_length 4
--------Dataset created--------
Dataset size:1371 entries
--------------Dataset has not been normalized yet. Perform data normalization and store the mean value of each column.--------------
--------------Max-value for each column [ 6.82480001 12.95160007 17.92740059  2.44950008]--------------
--------------Min-value for each column [ -7.04209995 -13.7730999   -5.28609991  -8.54819965]--------------
--------Normalized dataset loaded--------
--------Normalized dataset loaded--------
--------Normalized dataset loaded--------
Training dataset size: 986
Validation dataset size: 110
Testing dataset size: 275
Current Rule [0.5, 0.5]
Goto diffuse
Missing Rate 0.18180160466812545
self.eval_length 4
--------Dataset created--------
Dataset size:1371 entries
--------------Dataset has not been normalized yet. Perform data normalization and store the mean value of each column.--------------
--------------Max-

In [95]:
from scipy.stats import multivariate_normal
n_var = 3
min_corr = 0.1
max_corr = 0.3

mu = np.zeros(n_var)  # mean vector
np.random.seed(1)

corr = np.random.uniform(min_corr, max_corr, size=int(n_var * (n_var - 1) / 2))  # correlation vector (n_var, 2)
cov = np.zeros((n_var, n_var))  # covariance matrix

diag = np.eye(n_var)


cov[np.triu_indices(n_var, k=1)] = corr  # fill upper triangular part with correlations
cov = cov + cov.T + diag


n = 300  # sample size
np.random.seed(2)  # set seed so results are replicable
dat = multivariate_normal.rvs(mean=mu, cov=cov, size=n)  # data


In [79]:
import numpy as np

def missing(data, prob_miss, seed=100):
    np.random.seed(seed)
    m = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        m[i] = np.random.binomial(n=1, size=1, p=prob_miss[i])
    return m

In [80]:
import pandas as pd
import numpy as np

def miss_data(miss_ind, dat, miss_col, n_var):
    miss_dat = pd.DataFrame(dat)
    miss_dat['miss.ind'] = miss_ind
    miss_dat[miss_col] = np.where(miss_ind == 1, np.nan, miss_dat[miss_col])
    
    colnames = ['obs. var' + str(i) for i in range(1, n_var)] + ['miss.ind', 'miss.val']
    miss_dat.columns = colnames
    miss_dat.columns.values[miss_col] = 'miss.var'
    
    return miss_dat

In [141]:
def diffuse_mnar(target_miss, up_percentile, obs_percentile, dat, miss_col, n_var):
    missvar_bounds = np.quantile(dat[:, miss_col], up_percentile)
    temp = dat[:, miss_col] > missvar_bounds

    obsvar_bounds = np.quantile(dat[temp, :-1][:, 0], obs_percentile)

    miss_ind = np.zeros(len(dat))
    miss_ind[temp] = dat[temp, :-1][:, 0] > obsvar_bounds
    print(miss_ind)

    miss_dat = np.column_stack((dat, miss_ind, dat[:, miss_col]))
    miss_dat[miss_ind == 1, miss_col] = np.nan

    colnames = ['obs. var' + str(i) for i in range(1, n_var)] + ['miss. ind', 'miss. val']
    miss_dat = pd.DataFrame(miss_dat)
    #miss_dat.columns.values[miss_col] = 'miss.var'

    return miss_dat,dat

In [202]:
def diffuse_mnar(target_miss, up_percentile, obs_percentile, dat, miss_col, n_var):
    missvar_bounds = np.quantile(dat[:, miss_col], up_percentile)
    temp = dat[:, miss_col] > missvar_bounds

    obsvar_bounds = np.quantile(dat[temp, :-1][:, 0], obs_percentile)

    mask = np.zeros(len(dat))
    mask[temp] = dat[temp, :-1][:, 0] > obsvar_bounds


    return mask.sum()


In [203]:
diffuse_mnar(target_miss, up_percentile, obs_percentile, dat, miss_col, n_var)

60.0

In [10]:
target_miss = 0.2
up_percentile = 0.6  # 必须小于1 - target.miss
obs_percentile = 1 - target_miss/(1-up_percentile)

miss_col =2

In [80]:
def diffuse_mnar_single(data, up_percentile=0.5, obs_percentile=0.5):
    
    def scale_data(data):
      min_vals = np.min(data, axis=0)
      max_vals = np.max(data, axis=0)
      scaled_data = (data - min_vals) / (max_vals - min_vals)
      return scaled_data

    data = scale_data(data)

    mask = np.ones(data.shape)

    n_cols = data.shape[1]
    n_miss_cols = int(n_cols * 0.5)  # 选择50%的列作为缺失列
    miss_cols = np.random.choice(n_cols, size=n_miss_cols, replace=False)  # 随机选择缺失列的索引

    obs_cols = [col for col in range(data.shape[1]) if col not in miss_cols]
    for miss_col in miss_cols:
      print(miss_col)
      missvar_bounds = np.quantile(data[:, miss_col], up_percentile)
      print(data[:, miss_col])
      print(missvar_bounds)
      temp = data[:, miss_col] > missvar_bounds
      print(temp)


      obsvar_bounds = np.quantile(data[temp, :][:,obs_cols], obs_percentile)
      print(obsvar_bounds)
      
        # 初始化与原始数据维度相同的mask，所有元素均为1
      #mask[temp, miss_col] = (data[temp, :][:,obs_cols][:, 0] <= obsvar_bounds).astype(int)
      temp2 = data[:, miss_col]> obsvar_bounds
      #temp2 = data[:, miss_col] > missvar_bounds  # 根据缺失值情况将对应位置的值设为0
      print(temp2)
      print()
      # print(mask)
      # print()
    return mask

In [131]:
def diffuse_mnar_single(data, up_percentile = 0.5, obs_percentile = 0.5):
    
    def scale_data(data):
        min_vals = np.min(data, axis=0)
        max_vals = np.max(data, axis=0)
        scaled_data = (data - min_vals) / (max_vals - min_vals)
        return scaled_data

    data = scale_data(data)

    mask = np.ones(data.shape)

    n_cols = data.shape[1]
    n_miss_cols = int(n_cols * 0.5)  # 选择50%的列作为缺失列
    miss_cols = np.random.choice(n_cols, size=n_miss_cols, replace=False)  # 随机选择缺失列的索引

    obs_cols = [col for col in range(data.shape[1]) if col not in miss_cols]
    
    for miss_col in miss_cols:
        missvar_bounds = np.quantile(data[:, miss_col], up_percentile)
        temp = data[:, miss_col] > missvar_bounds
        
        obsvar_bounds = np.quantile(data[temp][:, obs_cols], obs_percentile)
        temp2 = data[:, miss_col] > obsvar_bounds

        merged_temp = np.logical_or(temp, temp2).astype(int)
        mask[:, miss_col] = merged_temp
    print("Missing Rate",1 - np.count_nonzero(mask) / mask.size)
    return mask

In [148]:
diffuse_mnar_single(dat,0.75,0.75)

0.23111111111111116


array([[1., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [1., 1

In [126]:
def diffuse_mnar_single(data, target_miss_rate, up_percentile, obs_percentile):
    
    def scale_data(data):
        min_vals = np.min(data, axis=0)
        max_vals = np.max(data, axis=0)
        scaled_data = (data - min_vals) / (max_vals - min_vals)
        return scaled_data
    
    def compute_miss_rate(mask):
        return 1 - np.count_nonzero(mask) / mask.size
    
    data = scale_data(data)

    mask = np.ones(data.shape)

    n_cols = data.shape[1]
    n_miss_cols = int(n_cols * 0.5)  # 选择50%的列作为缺失列
    miss_cols = np.random.choice(n_cols, size=n_miss_cols, replace=False)  # 随机选择缺失列的索引

    obs_cols = [col for col in range(data.shape[1]) if col not in miss_cols]
    
    while True:
        for miss_col in miss_cols:
            missvar_bounds = np.quantile(data[:, miss_col], up_percentile)
            temp = data[:, miss_col] > missvar_bounds
            obsvar_bounds = np.quantile(data[temp][:, obs_cols], obs_percentile)
            temp2 = data[:, miss_col] > obsvar_bounds

            merged_temp = np.logical_or(temp, temp2).astype(int)
            mask[:, miss_col] = merged_temp
        
        miss_rate = compute_miss_rate(mask)

        print("Missing Rate",miss_rate)
        
        if miss_rate <= target_miss_rate:
            #up_percentile += 0.05
            obs_percentile += 0.05
            print(up_percentile, obs_percentile)
        else:
            #print(up_percentile,obs_percentile)
            return mask

In [112]:
data = np.array([[10040,10001,10002],[1,2,1],[10,20,50],[1,1,2]]).T

In [129]:
mask = diffuse_mnar_single(dat,0.2,0.5,0.5)
mask

Missing Rate 0.15888888888888886
0.5 0.55
Missing Rate 0.16666666666666663
0.5 0.6000000000000001
Missing Rate 0.16666666666666663
0.5 0.6500000000000001
Missing Rate 0.16666666666666663
0.5 0.7000000000000002
Missing Rate 0.16666666666666663
0.5 0.7500000000000002
Missing Rate 0.16666666666666663
0.5 0.8000000000000003
Missing Rate 0.16666666666666663
0.5 0.8500000000000003
Missing Rate 0.16666666666666663
0.5 0.9000000000000004
Missing Rate 0.16666666666666663
0.5 0.9500000000000004
Missing Rate 0.16666666666666663
0.5 1.0000000000000004


ValueError: Quantiles must be in the range [0, 1]

0.033333333333333326
0.1 0.8


array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [1., 1

2
[0.   0.25 1.  ]
0.39999999999999997
[False False  True]
0.5
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]

0
[1.         0.         0.02564103]
0.22051282051282045
[ True False False]
0.0
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]



array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])